# Recommendation System

In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [26]:
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [27]:
post_data = pd.read_csv(r"Data\post_data.csv")
user_data = pd.read_csv(r"Data\user_data.csv")
view_data = pd.read_csv(r"Data\view_data.csv")

In [28]:
temp_df = pd.DataFrame(view_data)
# Make NumLikes for KNN
temp_df["NumLikes"] = np.random.randint(1, 11, len(temp_df))
display(temp_df.head())
display(temp_df.shape)

,user_id,post_id,time_stamp,NumLikes
0,5eece14ffc13ae660900008b,136781766,01/01/2019 01:30 PM,3
1,5eece14efc13ae660900003c,43094523,01/01/2019 01:33 PM,1
2,5eece14efc13ae6609000025,42428071,01/01/2019 01:43 PM,6
3,5eece14ffc13ae66090001d4,76472880,01/01/2019 01:54 PM,2
4,5eece14ffc13ae66090000ac,202721843,01/01/2019 02:00 PM,10


(71800, 4)

In [29]:
df = pd.merge(temp_df, post_data ,on='post_id')

In [30]:
df.category.unique()

array(['banking', 'programming', 'drawing', 'politics', 'zoology', 'art',
       'graphic', 'Music', 'Mathematics', 'photography', 'GST',
       'political', 'business', 'dance', 'HR management',
       'operating system', 'Craft', 'painting', 'Fashion Design',
       'science'], dtype=object)

In [31]:
# Create new more detailed post categories
df['category'] = df['category'].replace({'banking':'Mystery,Educational,Cyberpunk',
                                         'programming':'Fantasy,Magical-realism,Western',
                                         'drawing':'Sci-fi,Steamy,Alternate-history',
                                         'politics':'Romance,Young-adult,Adventure,Fantasy',
                                         'zoology':'Kids,Western,Inspirational,Mystery',
                                         'art':'Educational,Drama,Kids',
                                         'graphic':'Inspirational,Romance,Magical-realism',
                                         'Music':'Alternate-history,Adventure,Satire,Western',
                                         'Mathematics':'Cyberpunk,Steamy,Satire,Drama',
                                         'photography':'Magical realism,Adventure,Fantasy,Action',
                                         'GST':'Historical Fiction,Young-adult,Realistic-fiction,Drama',
                                         'political':'Realistic Fiction,Drama,Adventure,Satire',
                                         'business':'Horror,Action,Mystery,Cyberpunk',
                                         'dance':'Drama,Horror,Magical-realism',
                                         'HR management':'Western,Sci-fi,Fantasy',
                                         'operating system':'Action,Kids,Educational,Inspirational',
                                         'Craft':'Adventure,Inspirational,Sci-fi,Mystery,Young-adult',
                                         'painting':'Steamy,Alternate-history,Adventure',
                                         'Fashion Design':'Young Adult,Educational,Romance',
                                         'science':'Satire,Inspirational,Action,Fantasy'})

In [32]:
# timestamp not needed
data = df.drop(['time_stamp'], axis=1)
# drop duplicate tites to make the df managable 
#data = data.drop_duplicates(['title'])
display(data.head())
display(data.shape)

,user_id,post_id,NumLikes,title,category
0,5eece14ffc13ae660900008b,136781766,3,Sexy BANKING,"Mystery,Educational,Cyberpunk"
1,5eece14ffc13ae66090001da,136781766,7,Sexy BANKING,"Mystery,Educational,Cyberpunk"
2,5eece14efc13ae660900003a,136781766,8,Sexy BANKING,"Mystery,Educational,Cyberpunk"
3,5eece14ffc13ae6609000131,136781766,2,Sexy BANKING,"Mystery,Educational,Cyberpunk"
4,5eece14ffc13ae6609000167,136781766,10,Sexy BANKING,"Mystery,Educational,Cyberpunk"


(71800, 5)

In [35]:
combine_post_rating = data.dropna(axis = 0, subset = ['title'])

post_ratingCount = (combine_post_rating.
     groupby(by = ['title'])['NumLikes'].
     count().
     reset_index().
     rename(columns = {'NumLikes': 'TotalValue'})
     [['title', 'TotalValue']]
    )
post_ratingCount.head()

,title,TotalValue
0,10 Funny ART Quotes,15
1,10 Funny BANKING Quotes,10
2,10 Funny BUSINESS Quotes,13
3,10 Funny CRAFT Quotes,9
4,10 Funny DANCE Quotes,19


In [36]:
rating_with_TotalValue = combine_post_rating.merge(post_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_TotalValue.head()

,user_id,post_id,NumLikes,title,category,TotalValue
0,5eece14ffc13ae660900008b,136781766,3,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
1,5eece14ffc13ae66090001da,136781766,7,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
2,5eece14efc13ae660900003a,136781766,8,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
3,5eece14ffc13ae6609000131,136781766,2,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
4,5eece14ffc13ae6609000167,136781766,10,Sexy BANKING,"Mystery,Educational,Cyberpunk",18


In [37]:
popularity_threshold = 13
rating_popular_post = rating_with_TotalValue.query('TotalValue >= @popularity_threshold')
display(rating_popular_post.tail())
display(rating_popular_post.shape)

,user_id,post_id,NumLikes,title,category,TotalValue
71697,5eece14ffc13ae66090001bb,36975738,2,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71698,5eece14ffc13ae660900012d,36975738,7,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71699,5eece14efc13ae660900006c,36975738,4,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71700,5eece14ffc13ae660900007e,36975738,4,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71701,5eece14efc13ae6609000048,36975738,7,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13


(38292, 6)

In [38]:
rating_popular_post = rating_popular_post.drop_duplicates(['user_id', 'title'])
rating_popular_post_pivot = rating_popular_post.pivot(index = 'title', columns = 'user_id', values = 'NumLikes').fillna(0)
rating_popular_post_matrix = csr_matrix(rating_popular_post_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_popular_post_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [45]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(rating_popular_post_matrix, rating_popular_post_matrix)
np.save("cosine_sim_NumLikes.npy", cosine_sim)

# Test

In [47]:
query_index = np.random.choice(rating_popular_post_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(rating_popular_post_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

449


In [48]:
rating_popular_post_pivot.index[query_index]

' Apply These 5 Secret Techniques To Improve HR MANAGEMENT'

In [49]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_popular_post_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_post_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for  Apply These 5 Secret Techniques To Improve HR MANAGEMENT:

1:  3 BANKING Secrets You Never Knew, with distance of 0.6747907629434624:
2:  7 Ways To Keep Your DRAWING Growing Without Burning The Midnight Oil, with distance of 0.6756804703909008:
3:  Clear And Unbiased Facts About ART (Without All the Hype), with distance of 0.7066264206167643:
4:  3 POLITICAL Secrets You Never Knew, with distance of 0.7246659923735521:
5:  5 Best Ways To Sell BANKING, with distance of 0.7321433159062913:
